# Preprocessing 

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
sys.path.append("../src/")

In [3]:
from utilities import *

In [4]:
pd.set_option("display.max_columns", None)

### One Hot Encoding 

In [5]:
filepath = os.path.join(dir_dict["feature_engineered"], f"completed_feateng.parquet")
df = pd.read_parquet(filepath)

In [6]:
df[["fighter_name", "opponent_name"]] = df[["fighter_name", "opponent_name"]].astype("category")

In [7]:
df.select_dtypes(exclude=["number", "bool"])

,event_name,event_date,event_city,event_state,event_country,fight_bout,fight_method,fight_time_format,fight_referee,fight_details,fight_weight_class,fighter_name,fighter_nickname,fighter_stance,fighter_dob,opponent_name,opponent_nickname,opponent_stance,opponent_dob
0,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,open_weight_bout,decision_unanimous,1 Rnd (18),John McCarthy,<NA>,open_weight,Dan Severn,The Beast,southpaw,1958-06-08,David Abbott,Tank,switch,NaT
1,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,ultimate_ultimate_95_tournament_title_bout,decision_unanimous,1 Rnd + OT (27-3),John McCarthy,<NA>,open_weight,Dan Severn,The Beast,southpaw,1958-06-08,Oleg Taktarov,The Russian Bear,orthodox,1967-08-26
2,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,open_weight_bout,submission,1 Rnd (15),John McCarthy,Ankle Lock From Inoki-Ali Position,open_weight,Dave Beneteau,Dangerous,orthodox,NaT,Oleg Taktarov,The Russian Bear,orthodox,1967-08-26
3,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,open_weight_bout,submission,1 Rnd (15),John McCarthy,Rear Naked Choke,open_weight,Keith Hackney,The Giant Killer,sideways,NaT,Marco Ruas,King of the Streets,orthodox,1961-01-23
4,UFC - Ultimate Ultimate '95,1995-12-16,Denver,Colorado,USA,ultimate_ultimate_95_tournament_title_bout,decision_unanimous,1 Rnd + OT (27-3),John McCarthy,<NA>,open_weight,Oleg Taktarov,The Russian Bear,orthodox,1967-08-26,Dan Severn,The Beast,southpaw,1958-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13475,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,middleweight_bout,decision_split,5 Rnd (5-5-5-5-5),Herb Dean,Derek Cleary ...,middleweight,Sean Strickland,<NA>,orthodox,1991-02-27,Jared Cannonier,The Killa Gorilla,switch,1984-03-16
13476,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,flyweight_bout,ko_tko,3 Rnd (5-5-5),Keith Peterson,Punch to Head At Distance,flyweight,Alessandro Costa,Nono,orthodox,1996-01-28,Amir Albazi,The Prince,orthodox,1993-10-27
13477,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,welterweight_bout,decision_unanimous,3 Rnd (5-5-5),Herb Dean,Adalaide Byrd ...,welterweight,Bryan Battle,Pooh Bear,orthodox,1994-09-21,Rinat Fakhretdinov,Gladiator,orthodox,1991-09-28
13478,UFC Fight Night: Cannonier vs. Strickland,2022-12-17,Las Vegas,Nevada,USA,lightweight_bout,decision_unanimous,3 Rnd (5-5-5),Mark Smith,Mike Bell ...,lightweight,Damir Ismagulov,Qazaq,orthodox,1991-02-03,Arman Tsarukyan,Ahalkalakets,orthodox,1996-10-11


In [8]:
to_drop = ["event_name", "event_date", "event_city", "event_state", "event_country",
           "fight_bout", "fight_details", "fighter_dob", "opponent_dob", 
           "fighter_name", "opponent_name", "fighter_nickname", "opponent_nickname"]

In [9]:
df = df.drop(to_drop, axis=1)

In [10]:
cat_cols = df.select_dtypes(exclude=["number", "bool"]).columns
cat_cols

Index(['fight_method', 'fight_time_format', 'fight_referee',
       'fight_weight_class', 'fighter_stance', 'opponent_stance'],
      dtype='object')

In [11]:
df[cat_cols] = df[cat_cols].astype("category")

In [12]:
df = pd.concat([df, pd.get_dummies(df[cat_cols])], axis=1)

In [13]:
df = df.drop(cat_cols, axis=1)

In [14]:
bool_cols = df.select_dtypes("bool").columns
df[bool_cols] = df[bool_cols].astype("uint8")

In [15]:
df = df.drop("fight_opponent_win", axis=1)

In [16]:
to_drop = [col for col in df.columns if "fight_" in col] 
to_drop.remove("fight_fighter_win")
df = df.drop(to_drop, axis=1)

In [17]:
all_cols = df.columns

In [18]:
wins_cols = [col for col in df.columns if "_wins" in col]
len(wins_cols)

46

In [19]:
losses_cols = [col for col in df.columns if "_losses" in col]
len(losses_cols)

46

In [20]:
winrate_cols = [col for col in df.columns if "_winrate" in col]
len(winrate_cols)

46

In [21]:
stats_cols = [col for col in df.columns if "_cummean" in col]
len(stats_cols)

648

In [22]:
len(set(all_cols).difference(wins_cols).difference(losses_cols).difference(winrate_cols))

673

In [23]:
df.isna().sum().sort_values()

fighter_id                                    0
fighter_rollsum1_losses                       0
fighter_rollsum1_wins                         0
fighter_opponent_stance_switch_winrate        0
fighter_opponent_stance_switch_losses         0
                                          ...  
fighter_cummean_round4_td_%               12451
opponent_opponents_cummean_round5_td_%    12508
fighter_opponents_cummean_round5_td_%     12508
opponent_cummean_round5_td_%              12621
fighter_cummean_round5_td_%               12621
Length: 811, dtype: int64

In [24]:
null_counter = df.isna().sum()

null_pct = null_counter.sort_values() / df.shape[0]

In [25]:
too_much_missing = null_pct[null_pct > 0.5].index.to_list()

In [26]:
len(too_much_missing)

216

Round4 or Round5 columns

In [27]:
round_4_5_cols = [col for col in too_much_missing if "round4" in col or "round5" in col]
len(round_4_5_cols)

216

In [28]:
df = df.drop(round_4_5_cols, axis=1)

In [29]:
null_counter = df.isna().sum()

null_pct = null_counter.sort_values() / df.shape[0]

In [30]:
null_pct

fighter_id                                0.000000
fighter_rollsum1_losses                   0.000000
fighter_rollsum1_wins                     0.000000
fighter_opponent_stance_switch_winrate    0.000000
fighter_opponent_stance_switch_losses     0.000000
                                            ...   
opponent_cummean_round2_td_%              0.379896
fighter_opponents_cummean_round3_td_%     0.412315
opponent_opponents_cummean_round3_td_%    0.412315
opponent_cummean_round3_td_%              0.426929
fighter_cummean_round3_td_%               0.426929
Length: 595, dtype: float64

In [31]:
desc_cols = ["fighter_height_inches", "opponent_height_inches", "fighter_age", "opponent_age", "fighter_reach_inches",
             "opponent_reach_inches"]
df = df.loc[df[desc_cols].notna().all(axis=1)].reset_index(drop=True)

In [32]:
null_counter = df.isna().sum()

null_pct = null_counter.sort_values() / df.shape[0]

null_cols = null_pct[null_pct > 0].index.to_list()

In [33]:
len(null_cols)

432

In [34]:
len([col for col in null_cols if "cummean" in col])

432

#### Fighter Stats Columns

In [35]:
fighter_stat_cols = [col for col in null_cols if "fighter" in col]

In [36]:
df[fighter_stat_cols].isna().sum()

fighter_opponents_cummean_overall_td_landed    1415
fighter_cummean_round1_distance_ss_total       1415
fighter_cummean_round1_body_ss_landed          1415
fighter_cummean_overall_body_ss_total          1415
fighter_cummean_overall_body_ss_landed         1415
                                               ... 
fighter_cummean_round1_td_%                    3067
fighter_opponents_cummean_round2_td_%          3366
fighter_cummean_round2_td_%                    3697
fighter_opponents_cummean_round3_td_%          4060
fighter_cummean_round3_td_%                    4221
Length: 216, dtype: int64

In [37]:
df[fighter_stat_cols] = \
    df.groupby("fighter_id")[fighter_stat_cols].transform(lambda x: x.fillna(method="ffill").fillna(method="bfill"))

In [38]:
df[fighter_stat_cols].isna().sum()

fighter_opponents_cummean_overall_td_landed      91
fighter_cummean_round1_distance_ss_total         91
fighter_cummean_round1_body_ss_landed            91
fighter_cummean_overall_body_ss_total            91
fighter_cummean_overall_body_ss_landed           91
                                               ... 
fighter_cummean_round1_td_%                    1046
fighter_opponents_cummean_round2_td_%          1051
fighter_cummean_round2_td_%                    1529
fighter_opponents_cummean_round3_td_%          1493
fighter_cummean_round3_td_%                    1881
Length: 216, dtype: int64

In [47]:
df[fighter_stat_cols] = df[fighter_stat_cols].fillna(df[fighter_stat_cols].mean())

In [48]:
df[fighter_stat_cols].isna().sum()

fighter_opponents_cummean_overall_td_landed    0
fighter_cummean_round1_distance_ss_total       0
fighter_cummean_round1_body_ss_landed          0
fighter_cummean_overall_body_ss_total          0
fighter_cummean_overall_body_ss_landed         0
                                              ..
fighter_cummean_round1_td_%                    0
fighter_opponents_cummean_round2_td_%          0
fighter_cummean_round2_td_%                    0
fighter_opponents_cummean_round3_td_%          0
fighter_cummean_round3_td_%                    0
Length: 216, dtype: int64

#### Opponent Stats Columns

In [50]:
opponent_stat_cols = [col for col in null_cols if "opponent_" in col]

In [51]:
df[opponent_stat_cols].isna().sum()

opponent_cummean_round1_ground_ss_landed     1415
opponent_cummean_overall_ground_ss_total     1415
opponent_cummean_overall_ground_ss_landed    1415
opponent_cummean_round1_clinch_ss_total      1415
opponent_cummean_overall_clinch_ss_total     1415
                                             ... 
opponent_cummean_round1_td_%                 3067
opponent_opponents_cummean_round2_td_%       3366
opponent_cummean_round2_td_%                 3697
opponent_opponents_cummean_round3_td_%       4060
opponent_cummean_round3_td_%                 4221
Length: 216, dtype: int64

In [52]:
df[opponent_stat_cols] = \
    df.groupby("opponent_id")[opponent_stat_cols].transform(lambda x: x.fillna(method="ffill").fillna(method="bfill"))

In [53]:
df[opponent_stat_cols].isna().sum()

opponent_cummean_round1_ground_ss_landed       91
opponent_cummean_overall_ground_ss_total       91
opponent_cummean_overall_ground_ss_landed      91
opponent_cummean_round1_clinch_ss_total        91
opponent_cummean_overall_clinch_ss_total       91
                                             ... 
opponent_cummean_round1_td_%                 1046
opponent_opponents_cummean_round2_td_%       1051
opponent_cummean_round2_td_%                 1529
opponent_opponents_cummean_round3_td_%       1493
opponent_cummean_round3_td_%                 1881
Length: 216, dtype: int64

In [54]:
df[opponent_stat_cols] = df[opponent_stat_cols].fillna(df[opponent_stat_cols].mean())

In [55]:
df[opponent_stat_cols].isna().sum()

opponent_cummean_round1_ground_ss_landed     0
opponent_cummean_overall_ground_ss_total     0
opponent_cummean_overall_ground_ss_landed    0
opponent_cummean_round1_clinch_ss_total      0
opponent_cummean_overall_clinch_ss_total     0
                                            ..
opponent_cummean_round1_td_%                 0
opponent_opponents_cummean_round2_td_%       0
opponent_cummean_round2_td_%                 0
opponent_opponents_cummean_round3_td_%       0
opponent_cummean_round3_td_%                 0
Length: 216, dtype: int64

In [56]:
df.isna().sum().sort_values()

fighter_id                                           0
opponent_cummean_round3_head_ss_total                0
opponent_cummean_round3_body_ss_landed               0
opponent_cummean_round3_body_ss_total                0
opponent_cummean_round3_leg_ss_landed                0
                                                    ..
fighter_opponents_cummean_round2_ground_ss_landed    0
fighter_opponents_cummean_round2_ground_ss_total     0
fighter_opponents_cummean_round3_sig_str_landed      0
fighter_opponents_cummean_round3_total_str_landed    0
opponent_stance_switch                               0
Length: 595, dtype: int64

In [64]:
df.head()

,fight_fighter_win,fighter_height_inches,fighter_reach_inches,fighter_age,fighter_cumsum_wins,fighter_cumsum_losses,fighter_cumsum_winrate,fighter_cummean_overall_kd,fighter_cummean_round1_kd,fighter_cummean_round2_kd,fighter_cummean_overall_sig_str_%,fighter_cummean_round1_sig_str_%,fighter_cummean_round2_sig_str_%,fighter_cummean_overall_td_%,fighter_cummean_round1_td_%,fighter_cummean_round2_td_%,fighter_cummean_overall_sub_att,fighter_cummean_round1_sub_att,fighter_cummean_round2_sub_att,fighter_cummean_overall_rev,fighter_cummean_round1_rev,fighter_cummean_round2_rev,fighter_cummean_overall_sig_str_%_ss,fighter_cummean_round1_sig_str_%_ss,fighter_cummean_round2_sig_str_%_ss,fighter_cummean_round3_kd,fighter_cummean_round3_sig_str_%,fighter_cummean_round3_td_%,fighter_cummean_round3_sub_att,fighter_cummean_round3_rev,fighter_cummean_round3_sig_str_%_ss,fighter_cummean_overall_sig_str_landed,fighter_cummean_overall_sig_str_total,fighter_cummean_round1_sig_str_landed,fighter_cummean_round1_sig_str_total,fighter_cummean_round2_sig_str_landed,fighter_cummean_round2_sig_str_total,fighter_cummean_overall_total_str_landed,fighter_cummean_overall_total_str_total,fighter_cummean_round1_total_str_landed,fighter_cummean_round1_total_str_total,fighter_cummean_round2_total_str_landed,fighter_cummean_round2_total_str_total,fighter_cummean_overall_td_landed,fighter_cummean_overall_td_total,fighter_cummean_round1_td_landed,fighter_cummean_round1_td_total,fighter_cummean_round2_td_landed,fighter_cummean_round2_td_total,fighter_cummean_overall_sig_str_ss_landed,fighter_cummean_overall_sig_str_ss_total,fighter_cummean_round1_sig_str_ss_landed,fighter_cummean_round1_sig_str_ss_total,fighter_cummean_round2_sig_str_ss_landed,fighter_cummean_round2_sig_str_ss_total,fighter_cummean_overall_head_ss_landed,fighter_cummean_overall_head_ss_total,fighter_cummean_round1_head_ss_landed,fighter_cummean_round1_head_ss_total,fighter_cummean_round2_head_ss_landed,fighter_cummean_round2_head_ss_total,fighter_cummean_overall_body_ss_landed,fighter_cummean_overall_body_ss_total,fighter_cummean_round1_body_ss_landed,fighter_cummean_round1_body_ss_total,fighter_cummean_round2_body_ss_landed,fighter_cummean_round2_body_ss_total,fighter_cummean_overall_leg_ss_landed,fighter_cummean_overall_leg_ss_total,fighter_cummean_round1_leg_ss_landed,fighter_cummean_round1_leg_ss_total,fighter_cummean_round2_leg_ss_landed,fighter_cummean_round2_leg_ss_total,fighter_cummean_overall_distance_ss_landed,fighter_cummean_overall_distance_ss_total,fighter_cummean_round1_distance_ss_landed,fighter_cummean_round1_distance_ss_total,fighter_cummean_round2_distance_ss_landed,fighter_cummean_round2_distance_ss_total,fighter_cummean_overall_clinch_ss_landed,fighter_cummean_overall_clinch_ss_total,fighter_cummean_round1_clinch_ss_landed,fighter_cummean_round1_clinch_ss_total,fighter_cummean_round2_clinch_ss_landed,fighter_cummean_round2_clinch_ss_total,fighter_cummean_overall_ground_ss_landed,fighter_cummean_overall_ground_ss_total,fighter_cummean_round1_ground_ss_landed,fighter_cummean_round1_ground_ss_total,fighter_cummean_round2_ground_ss_landed,fighter_cummean_round2_ground_ss_total,fighter_cummean_round3_sig_str_landed,fighter_cummean_round3_sig_str_total,fighter_cummean_round3_total_str_landed,fighter_cummean_round3_total_str_total,fighter_cummean_round3_td_landed,fighter_cummean_round3_td_total,fighter_cummean_round3_sig_str_ss_landed,fighter_cummean_round3_sig_str_ss_total,fighter_cummean_round3_head_ss_landed,fighter_cummean_round3_head_ss_total,fighter_cummean_round3_body_ss_landed,fighter_cummean_round3_body_ss_total,fighter_cummean_round3_leg_ss_landed,fighter_cummean_round3_leg_ss_total,fighter_cummean_round3_distance_ss_landed,fighter_cummean_round3_distance_ss_total,fighter_cummean_round3_clinch_ss_landed,fighter_cummean_round3_clinch_ss_total,fighter_cummean_round3_ground_ss_landed,fighter_cummean_round3_ground_ss_total,fighter_cummean_overall_ctrl_seconds,fighter_cummean_ro

In [57]:
df = df.drop(["fighter_id", "opponent_id"], axis=1)

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [61]:
def model_performance(df):
    df_num = df.select_dtypes(["number", "bool"])
    bool_cols = df_num.select_dtypes("bool").columns
    df_num[bool_cols] = df_num[bool_cols].astype("uint8")
    df_num = df_num.fillna(0)

    X, y = df_num.drop("fight_fighter_win", axis=1), df_num["fight_fighter_win"]

    return np.mean(cross_val_score(LogisticRegression(random_state=42), X, y, cv=5, n_jobs=cpu_count()))

In [62]:
model_performance(df)

/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

0.608913518230086